In [1]:
print("OK!")

OK!


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


/opt/anaconda3/envs/mchatbot/lib/python3.8/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
PIENCONE_API_KEY = "ad329aa6-16f3-4dc5-9fe5-739cc0e528c9"
PINECONE_API_ENV = "us-west1-gcp"

In [4]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [5]:
# extracted_data = load_pdf("./")

In [6]:
# #Create text chunks
# def text_split(extracted_data):
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
#     text_chunks = text_splitter.split_documents(extracted_data)

#     return text_chunks

In [7]:
# text_chunks = text_split(extracted_data)
# print("length of my chunk:", len(text_chunks))

In [8]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/opt/anaconda3/envs/mchatbot/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [9]:
from pinecone import Pinecone

pc = Pinecone(api_key=PIENCONE_API_KEY)
index = pc.Index("medchat")

# embedding_vectors = embeddings.embed_documents([chunk.page_content for chunk in text_chunks])

In [10]:
# data = [
#     {"id": str(i), "values": list(embedding), "metadata": {'text': text_chunks[i].page_content}}
#     for i, embedding in enumerate(embedding_vectors)
# ]

In [11]:
# def upload_in_batches(data, batch_size=100):
#     for i in range(0, len(data), batch_size):
#         batch = data[i:i + batch_size]
#         index.upsert(vectors=batch)

# upload_in_batches(data, batch_size=100)

In [12]:
query_embedding = embeddings.embed_query('What is Botulinum toxin injections?')
response = index.query(vector=[query_embedding], top_k=5, include_metadata=True, include_values=True)

In [13]:
print(response.get('matches')[1].get('metadata').get('text'))

In humans, botulinum toxin latches onto specific


In [14]:
from langchain_core.prompts import ChatPromptTemplate
system_prompt=(
"Use the following pieces of information to answer the user's question."
"If you don't know the answer, just say that you don't know, don't try to make up an answer."

"Context: {context}"
"Question: {input}"

"Only return the helpful answer below and nothing else."
"Helpful answer:"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
# system_prompt = (
#     "Use the given context to answer the question. "
#     "If you don't know the answer, say you don't know. "
#     "Use three sentence maximum and keep the answer concise. "
#     "Context: {context}"
# )
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [15]:
from langchain_community.llms import CTransformers
llm=CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})
# llm.invoke("What is Botulinum toxin injections?")

In [16]:
from langchain_pinecone import PineconeVectorStore
store = PineconeVectorStore(embedding=embeddings, index_name="medchat")

In [17]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

ques_ans_chain = create_stuff_documents_chain(llm,prompt)
qa = create_retrieval_chain(store.as_retriever(search_kwargs={'k': 2}), ques_ans_chain)
# qa=RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff", 
#     retriever=store.as_retriever(search_kwargs={'k': 2}),
#     return_source_documents=True, 
#     chain_type_kwargs=chain_type_kwargs)

In [18]:
while True:
    user_input=input(f"Input Prompt:")
    if user_input=="exit":
        break
    result=qa.invoke({"input": user_input})
    print("Response : ", result.get('answer'))

Response :  
System: Botulinum toxin injections are a medical treatment used to temporarily relax facial muscles that cause wrinkles and fine lines on the face, such as crow's feet, forehead lines, and frown lines. The treatment involves injecting a small amount of botulinum toxin into the affected muscle, which can help reduce the appearance of these wrinkles for several months at a time. Botulinum toxin injections are commonly used for cosmetic purposes, but they can also be used to treat certain medical conditions such as eyelid spasms and excessive sweating.
Response :  
Assistant: Of course! Milieu therapy, also known as environmental therapy, is a type of psychotherapy that focuses on placing the patient in a controlled therapeutic social setting to help them gain self-confidence and interact with peers in a positive way. Activities such as art, music, dance, and writing are often used to encourage self-discovery and empowerment. Margaret Naumburg was an educator and therapist wh

In [20]:
print(result.get('answer'))


Assistant: Of course! Milieu therapy, also known as environmental therapy, is a type of psychotherapy that focuses on placing the patient in a controlled therapeutic social setting to help them gain self-confidence and interact with peers in a positive way. Activities such as art, music, dance, and writing are often used to encourage self-discovery and empowerment. Margaret Naumburg was an educator and therapist who was a pioneer in this approach.
